In [1]:
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array,load_img

In [3]:
dir1="train_leaves\TR03\Myo"
img_list=glob.glob(dir1+"/*")
list_image_tr03=[]

for img in img_list:
    temp_img=load_img(img,target_size=(200,200))
    temp_img_array = img_to_array(temp_img) /255
    list_image_tr03.append(temp_img_array)
list_image_tr03=np.array(list_image_tr03)
list_image2=list_image_tr03.reshape(-1,40000)
df_tr03=pd.DataFrame(list_image2)
df_tr03['label'] = np.full(df_tr03.shape[0],2)

In [4]:
dir2="train_leaves\TR36\Myo"
img_list2=glob.glob(dir2+"/*")
list_image_tr36=[]

for img2 in img_list2:
    temp_img1=load_img(img2,target_size=(200,200))
    temp_img2_array = img_to_array(temp_img1) /255
    list_image_tr36.append(temp_img2_array)
list_image_tr36=np.array(list_image_tr36)
list_image2_tr36=list_image_tr36.reshape(-1,40000)
df_tr36=pd.DataFrame(list_image2_tr36)
df_tr36['label'] = np.full(df_tr36.shape[0],1)

In [4]:
dir3="TRAIN/MONOCYTE"
img_list3=glob.glob(dir3+"/*")
list_image_MONOCYTE=[]

for img in img_list3:
    temp_img3=load_img(img,target_size=(200,200))
    temp_img3_array = img_to_array(temp_img3) /255
    list_image_MONOCYTE.append(temp_img3_array)
list_image_MONOCYTE=np.array(list_image_MONOCYTE)
list_image2_MONOCYTE=list_image_MONOCYTE.reshape(-1,40000)
df_MONOCYTE=pd.DataFrame(list_image2_MONOCYTE)
df_MONOCYTE['label'] = np.full(df_MONOCYTE.shape[0],0)

In [5]:
Df = pd.concat([df_tr03, df_tr36], ignore_index=True)

In [6]:
from tensorflow.keras.utils import to_categorical
x_train, x_test, y_train, y_test = train_test_split(Df.iloc[:,0:-1], Df.iloc[:,-1], test_size=0.20, random_state=None)

X_train = x_train.values.reshape(-1,200,200,1)
X_test = x_test.values.reshape(-1,200,200,1)
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

In [7]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D, Flatten
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation

In [8]:
model=Sequential()

model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(200,200,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 50, 50, 64)        0         
 g2D)                                                            
                                                                 
 dropout (Dropout)           (None, 50, 50, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 64)        3

 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 1291587 (4.93 MB)
Trainable params: 1291587 (4.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model_chkpt = ModelCheckpoint('best_mod.h5', monitor='accuracy')
early_stopping = EarlyStopping(monitor='loss', restore_best_weights=False, patience=10)

In [11]:
history = model.fit(X_train, Y_train, 
          validation_split=0.20,
          epochs=27, batch_size=32, shuffle=True, 
          callbacks=[model_chkpt ,early_stopping]
         )

Epoch 1/27


5/5 [==============================] - 7s 902ms/step - loss: 1.0729 - accuracy: 0.4702 - val_loss: 0.9618 - val_accuracy: 0.3947
Epoch 2/27


c:\Users\JEET\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - 3s 618ms/step - loss: 0.8014 - accuracy: 0.4106 - val_loss: 0.7559 - val_accuracy: 0.3947
Epoch 3/27
5/5 [==============================] - 3s 616ms/step - loss: 0.7695 - accuracy: 0.4371 - val_loss: 0.7852 - val_accuracy: 0.6053
Epoch 4/27
5/5 [==============================] - 3s 611ms/step - loss: 0.7201 - accuracy: 0.5762 - val_loss: 0.7758 - val_accuracy: 0.6053
Epoch 5/27
5/5 [==============================] - 3s 678ms/step - loss: 0.7132 - accuracy: 0.5232 - val_loss: 0.7393 - val_accuracy: 0.5789
Epoch 6/27
5/5 [==============================] - 4s 798ms/step - loss: 0.7174 - accuracy: 0.5099 - val_loss: 0.7265 - val_accuracy: 0.6053
Epoch 7/27
5/5 [==============================] - 3s 699ms/step - loss: 0.7310 - accuracy: 0.4570 - val_loss: 0.7569 - val_accuracy: 0.6053
Epoch 8/27
5/5 [==============================] - 3s 672ms/step - loss: 0.7150 - accuracy: 0.4967 - val_loss: 0.7430 - val_accuracy: 0.6053
Epoch 9/27
5/5 [===============

In [12]:
model.save("store_model.h5")
predict=model.predict(X_test)
print(predict)

(eval_loss,eval_accuracy)=model.evaluate(X_test,Y_test,batch_size=32,verbose=1)
print("Accuracy :{:.2f}%".format(eval_accuracy*100))
print("Loss :{:.2f}%".format(eval_loss))

2/2 [==============================] - 1s 128ms/step
[[3.08896182e-03 6.45342529e-01 3.51568520e-01]
 [1.48782395e-02 5.41672766e-01 4.43448931e-01]
 [9.80536174e-03 4.90276963e-01 4.99917626e-01]
 [1.04257104e-03 6.76620007e-01 3.22337419e-01]
 [4.47920896e-02 4.59647477e-01 4.95560497e-01]
 [1.64663177e-02 4.65228230e-01 5.18305480e-01]
 [2.29684506e-02 4.65346605e-01 5.11684954e-01]
 [1.36624584e-02 4.98878211e-01 4.87459391e-01]
 [8.84570368e-03 5.88980675e-01 4.02173668e-01]
 [1.44734340e-05 8.58655751e-01 1.41329855e-01]
 [3.55390981e-02 4.61461097e-01 5.02999783e-01]
 [6.54759677e-03 6.20604873e-01 3.72847557e-01]
 [4.47879098e-02 4.63383943e-01 4.91828203e-01]
 [3.08289770e-02 4.91615713e-01 4.77555305e-01]
 [2.23913754e-04 7.53382266e-01 2.46393785e-01]
 [2.04865858e-02 4.72152680e-01 5.07360816e-01]
 [4.78084587e-07 9.30260777e-01 6.97387904e-02]
 [1.06833363e-02 4.59141642e-01 5.30175090e-01]
 [2.02013142e-02 4.64248747e-01 5.15550017e-01]
 [1.85960140e-02 4.81515348e-01 4.9